In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

<h5>Rough Draft</h5>

Types of models to use :
* Linear Regression (Check if underfit with plots)
    - Log transform skewed distributions
* KNN
* XGBoost
* https://automl.github.io/auto-sklearn/master/

How to finetune model:
* param grid tuning
* Cross Validation
* K Best features
* R-squared, Adjusted R-squared

How to interpret model:
* Feature Importance
* Permutation Importance
* SHAP values and plots
* Partial Dependence Plots (1 and 2D)



GroupKFold (K = 4, Group by country, then remove country)

Pipeline (Scale, PCA)

In [ ]:
df = pd.read_csv('../data/interim/radius3_selected_features.csv',index_col=[0])
X = df.drop(['Gini index','Country Name'],axis=1).to_numpy()

y = df['Gini index'].to_numpy()
groups = df['Country Name'].to_numpy()

In [ ]:
df.head()

<h5>Preprocessing</h5>
Below I set up the initial pipeline that preprocesses the data and splits it into folds.

* since values between countries were imputed, Group K fold seperates the data and makes sure that countries don't overlap.
* Each fold is then standard scaled with the rest of the folds
* In a similar way I use PCA to reduce the 6 poverty features into 2

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

preprocess_steps = [('scaler',StandardScaler()),
                    ('ct',ColumnTransformer([('pca',PCA(n_components=2),list(range(13,19)))],remainder='passthrough'))]

In [ ]:
from sklearn.pipeline import Pipeline

Pipeline(steps=preprocess_steps)

In [ ]:
from sklearn.model_selection import GroupKFold

gkf = GroupKFold(n_splits=5)

<h5>Linear models</h5>

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score,cross_val_predict


ols = Pipeline(steps=preprocess_steps+[('ols',LinearRegression())])
test_scores = cross_val_score(ols,X,y,groups=groups,cv=gkf)

In [ ]:
np.average(test_scores)

In [ ]:
preds = cross_val_predict(ols,X,y,groups=groups,cv=gkf)

sns.jointplot(x=preds,y=preds-y)
plt.xlabel('Predicted Gini Index')
plt.ylabel('Residuals')
plt.show()


Since there's correlation in the residuals and Gini Index isn't normally distributed, Linear regression (or atleast ordinary least squares) isn't a good fit for the data.

* I next tried LassoLars which includes a regularization parameter to prevent overfitting and increases coefficents of features in a forward stepwise manner. 
* Although it doesn't change the fact that the data doesn't fit a linear model, it's coefficients give good insight into which features are most important and the direction of that relationship. 



In [ ]:
from sklearn.linear_model import LassoLars

lars = Pipeline(steps=preprocess_steps+[('lars',LassoLars())])

test_scores = cross_val_score(lars,X,y,groups=groups,cv=gkf)

In [ ]:
print(np.average(test_scores))

In [ ]:
columns = df.drop(['Country Name', 'Gini index'],axis=1).columns.to_list()
feature_names = [col for col in columns if col.find('Poverty') < 0]

feature_names = ['poverty_pca0','poverty_pca1'] + feature_names #the column transformer outputs features in this order

In [ ]:
folds = gkf.split(X,y,groups=groups)
coefs = {}
for f in feature_names:
    coefs[f] = []
for i in range(5):
    train_ind, test_ind = next(folds)
    X_train, y_train, X_test, y_test = X[train_ind], y[train_ind], X[test_ind], y[test_ind]
    lars.fit(X_train,y_train)
    fold_coefs = lars.named_steps['lars'].coef_
    
    for j in range(len(feature_names)):
        coefs[feature_names[j]].append(fold_coefs[j])

In [ ]:
sns.barplot(pd.DataFrame(coefs),orient='h')
plt.title('LassoLars coefficients');

Since all the variables are standard scaled, The coefficients can compared directly.

According to lasso the most important variables in order are:
- Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative)
- Cause of death, by injury (% of total)
- Hospital beds (per 1,000 people)  
- GDP per capita

<h5>K Neighbors Regression</h5>

In [ ]:

from sklearn.neighbors import KNeighborsRegressor

KNN = Pipeline(steps=preprocess_steps+[('knn',KNeighborsRegressor())])

test_scores = cross_val_score(KNN,X,y,groups=groups,cv=gkf)

In [ ]:
print(test_scores)

In [ ]:
np.average(test_scores)

In [ ]:
from sklearn.model_selection import cross_validate

avg_train_scores = []
avg_test_scores = []
nn_range = range(2,20)

for nn in nn_range:
    train_scores = []
    test_scores = []
    knn = KNN.set_params(knn__n_neighbors=nn)
    scores = cross_validate(knn,X,y,groups=groups,cv=gkf,return_train_score=True)
    
    avg_train_scores.append(np.average(scores['train_score']))
    avg_test_scores.append(np.average(scores['test_score']))

In [ ]:
plt.plot(nn_range,avg_train_scores) # type: ignore
plt.plot(nn_range,avg_test_scores) # type: ignore
plt.grid(True)
plt.xlabel('number of neighbors')
plt.ylabel('R-squared')
plt.legend({'train':'blue','test':'orange'})
plt.show()

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.pipeline import Pipeline

avg_train_scores = []
avg_test_scores = []

pipeline = Pipeline(steps=preprocess_steps+[('skb',SelectKBest()),('knn',KNeighborsRegressor())])

for k_val in range(1,16):
    
    model = pipeline.set_params(skb__k=k_val,skb__score_func=mutual_info_regression,knn__n_neighbors=8)

    scores = cross_validate(model,X,y,groups=groups,cv=gkf,return_train_score=True)
    
    avg_train_scores.append(np.average(scores['train_score']))
    avg_test_scores.append(np.average(scores['test_score']))
    

In [ ]:
plt.plot(range(1,16),avg_train_scores) # type: ignore
plt.plot(range(1,16),avg_test_scores) # type: ignore
plt.grid(True)
plt.xlabel("'k' best features")
plt.ylabel('R-squared')
plt.legend({'train':'blue','test':'orange'})
plt.show()


The dip in the middle shows that select K best features may not be selecting optimally

Next I run a grid search to tune the parameters for K nearest neighbors

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import f_regression,r_regression,mutual_info_regression

pipeline = Pipeline(steps=preprocess_steps+[('skb',SelectKBest()),('knn',KNeighborsRegressor())])

params = {
    'skb__k':range(1,16),                                                 #how many features to select
    'skb__score_func':[mutual_info_regression,f_regression,r_regression], #how to select features
    'knn__n_neighbors':range(5,15),                                       #number of neighbors
    'knn__weights':['uniform','distance'],                                #weigh all neighbors uniformly or by distance
    'knn__p':[1,2,3]                                                      # how to measure distance, manhattan, euclidean, etc.
}

grid_search = GridSearchCV(pipeline,params,cv=gkf)

grid_search.fit(X,y,groups=groups)

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.best_params_

So the best KNN model explains about 60% of the variance and uses 15 of the 16 features

In [ ]:
skb = SelectKBest(score_func=mutual_info_regression,k=15)

pipe = Pipeline(preprocess_steps)

X_transformed = pipe.fit_transform(X,y)

skb.fit(X_transformed,y)
best_features=skb.get_feature_names_out(feature_names) # type: ignore 

In [ ]:
best_features

In [ ]:
set(df.columns) - set(best_features)

The feature it omits is Year

<h5>XG Boost</h5>

In [ ]:
from xgboost import XGBRegressor

xgb = Pipeline(preprocess_steps + [('xgb',XGBRegressor())])

test_scores = cross_val_score(xgb,X,y,groups=groups,cv=gkf)

In [ ]:
print(test_scores)

In [ ]:
np.average(test_scores)

In [ ]:
folds = gkf.split(X,y,groups=groups)
feature_importances = {}
for f in feature_names:
    feature_importances[f] = []

for i in range(5):
    train_ind, test_ind = next(folds)
    X_train, y_train, X_test, y_test = X[train_ind], y[train_ind], X[test_ind], y[test_ind]
    xgb.fit(X_train,y_train)

    fold_fi = xgb.named_steps['xgb'].feature_importances_
    
    for j in range(len(feature_names)):
        feature_importances[feature_names[j]].append(fold_fi[j])
    




In [ ]:
pd.DataFrame(feature_importances)

In [ ]:
sns.barplot(pd.DataFrame(feature_importances),orient='h')

- Just like the lassolars model, XGBoost also indicates that Hospital beds per 1000 people, Cause of death by injury and lower secondary education as the most important features.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'xgb__n_estimators':np.arange(5,150,5),
    'xgb__max_depth':np.arange(2,9),
    'xgb__learning_rate':np.arange(0,1,0.2),
    'xgb__reg_lambda':np.arange(0,2,0.4),
    'xgb__reg_alpha':np.arange(0,1,0.2)
}

random_search = RandomizedSearchCV(xgb,param_distributions=param_grid,cv=gkf,n_iter=450)

random_search.fit(X,y,groups=groups)

In [ ]:
random_search.best_score_

In [ ]:
random_search.best_params_

<h5>K Neighbors vs XG Boost</h5>

- KNN scores a bit better with a score of 0.59 vs 0.56
- xgboost model is more interpretable as we can look at the feature importance.
- While KNN is less computationally expensive with training, it is more expensive for prediction since it requires storing the entire dataset 

Let's see if the residuals between both models can highlight any differences between them.

In [ ]:
knn_preds = cross_val_predict(grid_search.best_estimator_,X,y,groups=groups,cv=gkf)

sns.jointplot(x=knn_preds,y=knn_preds-y)
plt.xlabel('predicted Gini Index')
plt.ylabel('KNN Residuals')
plt.show()

In [ ]:
xgb_preds = cross_val_predict(random_search.best_estimator_,X,y,groups=groups,cv=gkf)

sns.jointplot(x=xgb_preds,y=xgb_preds-y)
plt.xlabel('predicted Gini Index')
plt.ylabel('XGBoost Residuals')
plt.show()

Both Models have similar residual graphs but XGBoosts residuals seem to be centered a bit lower than 0

another way to decide is to see which model performs better with less features

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector

xgb_params = random_search.best_params_
xgb_rfe = Pipeline(preprocess_steps + [('rfe',RFE(XGBRegressor())),('xgb',XGBRegressor())])

knn_params = grid_search.best_params_
del knn_params['skb__k']
del knn_params['skb__score_func']
best_knn = KNeighborsRegressor(n_neighbors=12,p=1,weights='uniform')
knn_sfs = Pipeline(preprocess_steps + [('sfs',SequentialFeatureSelector(best_knn)),('knn',KNeighborsRegressor())])

xgb_test_scores = []
knn_test_scores = []

for i in range(1,len(feature_names)):
    knn_model = knn_sfs.set_params(sfs__n_features_to_select=i,**knn_params)
    xgb_model = xgb_rfe.set_params(rfe__n_features_to_select=i,**xgb_params)

    xgb_test_scores.append(np.average(cross_val_score(xgb_model,X,y,groups=groups,cv=gkf)))
    knn_test_scores.append(np.average(cross_val_score(knn_model,X,y,groups=groups,cv=gkf)))


In [ ]:
plt.plot(range(1,len(feature_names)),xgb_test_scores) #type: ignore
plt.plot(range(1,len(feature_names)),knn_test_scores) #type: ignore
plt.xlabel('number of features')
plt.ylabel('R-squared')
plt.grid(True)
plt.legend({'xgb':'blue','knn':'orange'})
plt.show()

It looks like xgb is better with less features so that's the model I'll go with

Let's see which features we can drop

In [ ]:
from sklearn.feature_selection import RFE

rfe_params = {'reg_lambda': 1.2000000000000002,
 'reg_alpha': 0.2,
 'n_estimators': 115,
 'max_depth': 2,
 'learning_rate': 0.4}

folds = gkf.split(X,y,groups=groups)
scores = []
chosen_features = {}
for f in feature_names:
    chosen_features[f] = 0

for i in range(5):
    train_ind, test_ind = next(folds)
    X_train, y_train, X_test, y_test = X[train_ind], y[train_ind], X[test_ind], y[test_ind]
    preprocessor = Pipeline(steps=preprocess_steps)
    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)

    rfe = RFE(XGBRegressor(**rfe_params),n_features_to_select=10)
    rfe.fit(X_train,y_train)
    scores.append(rfe.score(X_test,y_test))
    selected = rfe.get_feature_names_out(feature_names) # type: ignore
    for f in feature_names:
        if f in selected:
            chosen_features[f] += 1

In [ ]:
pd.Series(chosen_features) #Number of times features were in the top ten over 5 folds

Looking above, we can eliminate:
- Year
- Inflation
- Unemployment
- Life expectancy
- Physicians

Through trial, error and looking at permutation importance, I was also able to drop:
- GDP (current US$) 
    - I investigated this as it can be calculated from population and GDP per capita
- Educational attainment, Doctoral or equivalent, population 25+, total (%) (cumulative)
    - showed negative permutation importance
- Educational attainment, at least Master's or equivalent, population 25+, total (%) (cumulative)
    - showed negative permutation importance
- Educational attainment, at least completed short-cycle tertiary, population 25+, total (%) (cumulative)
    - showed low permutation importance

In [ ]:
final_features = df.drop(['Inflation, consumer prices (annual %)',
                          'Unemployment, total (% of total labor force) (modeled ILO estimate)',
                          'Life expectancy at birth, total (years)',
                          'Physicians (per 1,000 people)','GDP (current US$)',"Educational attainment, at least Master's or equivalent, population 25+, total (%) (cumulative)",
                          "Educational attainment, Doctoral or equivalent, population 25+, total (%) (cumulative)",
                          "Educational attainment, at least completed short-cycle tertiary, population 25+, total (%) (cumulative)"],axis=1)

final_features.to_csv('../data/processed/selected_features.csv')